In [16]:
import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

import scipy.optimize

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import seaborn as sns

import window_detect2 as wd2

In [17]:
# simulation data 
sim_data_path = "../../data/energy_model/230307_match.pkl"
sim_data = pd.read_pickle(sim_data_path)

# smoothing on different time series than the observation 
sim_smooth = wd2.make_stl_smooth(series=sim_data["Always Closed"], time=None, model="additive")

# rename label in simulation dataframe to match class
sim_data.rename(columns={"Varied": "Temp C"}, inplace=True)

s1 = wd2.Window_Detect2(sim_data)
s1.analyze_window_change(smooth_fx=None, sim_smooth=sim_smooth)

# measured data 
a00, a01 = h.import_desired_data("A", "15T")

w1 = wd2.Window_Detect2(a01)
w1.analyze_window_change(wd2.make_stl_smooth)

w2 = wd2.Window_Detect2(a01)
w2.analyze_window_change(wd2.make_sin_smooth)

w3 = wd2.Window_Detect2(a01)
w3.analyze_window_change(wd2.make_ewm_smooth)



In [18]:
# b00, b01 = h.import_desired_data("B", "15T")
# print(b00["Window Open"].unique(), b01["Window Open"].unique())

# c00, c01 = h.import_desired_data("C", "15T")
# print(c00["Window Open"].unique(), c01["Window Open"].unique())

# b00, c00

In [36]:
def print_num_shift(data):
    return len(data[(data["Window Open"].shift() != data["Window Open"])])

In [1]:
def count_score(c, hits, near_miss, miss, near_miss_lim=2):
    if c == 0:
        hits+=1
    elif c <= near_miss_lim:
        near_miss+=1
    else:
        miss+=1
    
    return hits, near_miss, miss


def report_score_ratios(obj, z=2, timedelta=0, near_miss_lim=2):
    """ 
    obj: window_detect2 object, see notebooks/230613_metrics2.ipynb and scripts/window_detect2.py 
    z: use first or second z score 
    timedelta: length of time to include to be considered 1 hit 
    near_miss_lim: number of timesteps away that can be considered a near miss TODO make this and timedelta the same..
    """

    hit = 0 
    near_miss = 0 
    miss = 0
    
    obj.make_guesses(z=z, timedelta=timedelta)
    # print(len(obj.guess_times))
    # print(obj.guess_times.iloc[-1])

    res = pd.DataFrame(obj.guess_times).apply(lambda x: h.calc_win_change_dist(a01, x.name), axis=1)

    scores = res.apply(lambda x: count_score(np.abs(x[2]), hit, near_miss, miss, near_miss_lim))

    score_sum = pd.DataFrame(scores.to_list(), columns=["hit", "near_hit", "miss"]).sum()

    correct = score_sum["hit"] / len(scores) 
    almost_correct = (score_sum["hit"] +  score_sum["near_hit"])/ len(scores) 
    incorrect = score_sum["miss"]/ len(scores) 
    ratios = [correct, almost_correct, incorrect]
    ratio_nice = [100 * np.round(i, 2) for i in ratios]


    return score_sum, ratio_nice


In [37]:


all_scores = {}
exp_names = ["A", "B", "C"]
smooth_names = ["stl", "sin", "ewm"]

for data, name in zip([a01, b00, c00], exp_names):
    all_scores[name] = {}
    print("shift:",  print_num_shift(data))

    try:
        w1 = wd2.Window_Detect2(data)
        w1.analyze_window_change(wd2.make_stl_smooth)

        w2 = wd2.Window_Detect2(data)
        w2.analyze_window_change(wd2.make_sin_smooth)

        w3 = wd2.Window_Detect2(data)
        w3.analyze_window_change(wd2.make_ewm_smooth)

        objects = [w1, w2, w3,]

        for obj, sname in zip(objects, smooth_names):
            a, r = report_score_ratios(obj, z=1)
            all_scores[name][sname] = r[1]
    except:
        pass

shift: 13
shift: 8
shift: 8


In [38]:
all_scores

{'A': {'stl': 0.0, 'sin': 60.0, 'ewm': 38.0},
 'B': {},
 'C': {'stl': 5.0, 'sin': 25.0, 'ewm': 12.0}}

In [23]:
objects = [s1, w1, w2, w3,]

In [24]:
for obj in objects:
    a, r = report_score_ratios(obj, z=1)
    print(a, r[1], "\n")
    

hit          0
near_hit     1
miss        10
dtype: int64 9.0 

hit          0
near_hit     1
miss        18
dtype: int64 5.0 

hit         0
near_hit    2
miss        6
dtype: int64 25.0 

hit          1
near_hit     1
miss        14
dtype: int64 12.0 



In [25]:
for obj in objects:
    a, r = report_score_ratios(obj, z=2)
    print(a, r[1], "\n")

hit          0
near_hit     0
miss        16
dtype: int64 0.0 

hit          0
near_hit     0
miss        17
dtype: int64 0.0 

hit          1
near_hit     2
miss        13
dtype: int64 19.0 

hit          1
near_hit     2
miss        15
dtype: int64 17.0 



In [26]:
_, r = report_score_ratios(w1)
r[1]

0.0

In [27]:
w1.make_guesses(z=2, timedelta=0)

In [28]:
res = pd.DataFrame(w1.guess_times).apply(lambda x: h.calc_win_change_dist(a01, x.name), axis=1)


In [29]:
time = pd.Timedelta(minutes=15)
time

Timedelta('0 days 00:15:00')

In [30]:
res.apply(lambda x: x[2]*time)

46      0 days 03:45:00
47      0 days 03:30:00
49      0 days 03:00:00
77    -1 days +20:00:00
78    -1 days +19:45:00
143   -1 days +22:30:00
145   -1 days +22:00:00
238   -1 days +21:15:00
239   -1 days +21:00:00
241   -1 days +20:30:00
242     0 days 03:15:00
269   -1 days +20:30:00
270   -1 days +20:15:00
280   -1 days +17:45:00
281   -1 days +17:30:00
285   -1 days +16:30:00
286   -1 days +16:15:00
dtype: timedelta64[ns]

In [31]:
hits = 0
near_miss = 0
miss = 0

scores = res.apply(lambda x: count_score(np.abs(x[2]), hits, near_miss, miss))
scores


46     (0, 0, 1)
47     (0, 0, 1)
49     (0, 0, 1)
77     (0, 0, 1)
78     (0, 0, 1)
143    (0, 0, 1)
145    (0, 0, 1)
238    (0, 0, 1)
239    (0, 0, 1)
241    (0, 0, 1)
242    (0, 0, 1)
269    (0, 0, 1)
270    (0, 0, 1)
280    (0, 0, 1)
281    (0, 0, 1)
285    (0, 0, 1)
286    (0, 0, 1)
dtype: object

In [32]:
len(scores)

17

In [33]:
score_sum = pd.DataFrame(scores.to_list(), columns=["hit", "near_miss", "miss"]).sum()
score_sum

hit           0
near_miss     0
miss         17
dtype: int64

In [34]:
correct = score_sum["hit"] / len(scores)
almost_correct = score_sum["hit"] +  score_sum["near_miss"]/ len(scores)
incorrect = score_sum["miss"]/ len(scores)

print(correct, almost_correct, incorrect)

0.0 0.0 1.0


In [35]:
# hit = 0 
# near miss => abs(distance) < 2 => 30 minustes
# miss =>

